In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from pathlib import Path
import os
import glob
import numpy
from tqdm import tqdm

In [2]:
folder = 'D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-02-24/categories/*'


In [3]:
# Rename files in directory so that they have the correct file extension.
def converthtmlsr2(folder):
    """Changes file type to from xxxx to xxxx.html so that the files can be read in correctly"""
    for filename in glob.iglob(os.path.join(folder, 'ite*')):
      os.rename(filename, filename + '.html')

#converthtmlsr2(folder)

In [ ]:
# Coinbase API

In [68]:
def cleanhtmlsr2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Vendor Rating, Product Review Count, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  details_raw = raw_html.find_all('div', {'class' :'item_details'})
  details = re.sub(cleantags, '', str(details_raw)).strip().split('\n    ')
  sellerid_full = details[1::4]
  sellerid = [i.split(': ', 1)[1] for i in sellerid_full]
  origin_full = details[2::4]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = details[3::4]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' :'price_big'})
  priceBTC = str(price_raw).replace('[','').replace(']','').replace('<div class="price_big">฿','').replace('</div>','').split(', ')
  rating = []  # Seller Rating. Out of 5.
  reviews = [] # The number of customer reviews
  priceUSD = [] # Price in American Dollars
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  return df

#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/art/items.html')
#cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2013-12-20/categories/drugs-dissociatives-dioscorea/items.html')
cleanhtmlsr2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-01-16/categories/drugs-dissociatives-dioscorea/items.html')

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,"[28g / 1oz - Mega Sedative ""Wild Yam"" - Diosco...",b1g1mpact,None,0.026687,None,None,Czech Republic,Worldwide,Dissociatives,Dioscorea,SilkRoad2,2014-01-16


In [58]:
#works for final date

def cleanhtmlsr2part2(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Vendor Rating, Product Review Count, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  sellerid_raw = raw_html.find_all('div', {'class' :'vendor'})
  sellerid_full = [i.text for i in sellerid_raw]
  sellerid_name = [i.split('by ')[1] for i in sellerid_full]
  sellerid = [i.split('\n')[0] for i in sellerid_name]
  shipping_full = raw_html.find_all('div', {'class' : 'shipping'})
  shipping = [i.text for i in shipping_full]
  origin_full = [i.split('\n ')[1] for i in shipping]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = [i.split('\n ')[2] for i in shipping]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' : 'price'})
  priceBTC = [i.text for i in price_raw]
  priceBTC = [i.replace('฿','') for i in priceBTC]
  rating_full = raw_html.find_all('span', {'class' : 'vendor_rating average'})  # Seller Rating. Out of 5.
  rating = [i.split('\n')[2] for i in sellerid_full]
  for i in range(0, len(rating)): 
    try:
      float(rating[i])
    except ValueError:
      rating[i] = 0
  if rating != None:
    for i in range(0, len(rating)): 
      rating[i] = float(rating[i])/20
  reviews_full = raw_html.find_all('div', {'class' : 'rating_count'}) # The number of customer reviews
  reviews = [i.text for i in reviews_full]
  for i in range(0, len(reviews)): 
    reviews[i] = reviews[i].replace('(','').replace(')','')
    reviews[i] = int(reviews[i])
  priceUSD = [] # Price in American Dollars
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  return df

#cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-11-05/categories/art/items.html')
cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-10-04/categories/apparel/items_page=11.html')
#cleanhtmlsr2part2('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-10-04/categories/drugs-stimulants/items.html')

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,[Rolex - Lady Datejust 36mm Pink [Replica],RepAAA,None,0.416245,4.65,0,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-10-04
1,Hublot Maradona Replica,AsianVixen,None,0.513340,4.45,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
2,Prada Calf Leather Twin Pocket Glace Tote Bag ...,FoxyGirl,None,1.026680,4.5,0,China,Worldwide,Apparel,None,SilkRoad2,2014-10-04
3,IWC - Portuguese Chronograph RG Silver Dial [A...,RepAAA,None,1.114681,4.65,0,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-10-04
4,█GOLD FLEX BODY ARMOR █ CASUAL █CONCEALED█,LegalGoods,None,1.906691,0,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
5,Chanel Beige Ballerina Ballet Flats Shoes Repl...,FoxyGirl,None,0.528007,4.5,0,China,Worldwide,Apparel,None,SilkRoad2,2014-10-04
6,Hublot F1 Quartz Replica,AsianVixen,None,0.484006,4.45,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
7,Rolex Watches Women 187 ID: 43406,gotmilkreplica,None,0.396005,4.15,0,"Hong Kong, (China)",Worldwide,Apparel,None,SilkRoad2,2014-10-04
8,Rolex DateJust Zebra Replica,AsianVixen,None,0.528007,4.45,0,Undeclared,Worldwide,Apparel,None,SilkRoad2,2014-10-04
9,Custom Listing for W...h,FoxyGirl,None,0.542674,4.5,0,China,Worldwide,Apparel,None,SilkRoad2,2014-10-04


In [97]:
###Currently same as part 2

def cleanhtmlsr2part3(filename):
  """Extracts relevant information from SilkRoad2 html files.
  Resulting dataframe contains:
  Title, Seller ID, priceUSD, priceBTC, Vendor Rating, Product Review Count, Origin, Destination, Category, Subcategory, Market Name and Date Scraped"""
  with open(filename, encoding='utf8') as infile:  # Open file to support utf8 encoding
    raw_html = BeautifulSoup(infile, "lxml")
  title_raw = raw_html.find_all('div', {'class' :'item_title'})
  cleantags = re.compile('<.*?>')  # Regex for html tags
  title = re.sub(cleantags, '', str(title_raw)).strip().split(', ')  # Strip html tags
  sellerid_raw = raw_html.find_all('div', {'class' :'vendor'})
  sellerid_full = [i.text for i in sellerid_raw]
  sellerid_name = [i.split('by ')[1] for i in sellerid_full]
  sellerid = [i.split('\n')[0] for i in sellerid_name]
  shipping_full = raw_html.find_all('div', {'class' : 'shipping'})
  shipping = [i.text for i in shipping_full]
  origin_full = [i.split('\n ')[1] for i in shipping]
  origin = [i.split(': ', 1)[1] for i in origin_full]
  destination_full = [i.split('\n ')[2] for i in shipping]
  destination = [i.split(': ', 1)[1] for i in destination_full]
  price_raw = raw_html.find_all('div', {'class' : 'price'})
  priceBTC = [i.text for i in price_raw]
  priceBTC = [i.replace('฿','') for i in priceBTC]
  rating_full = raw_html.find_all('span', {'class' : 'vendor_rating average'})  # Seller Rating. Out of 5.
  rating = [i.split('\n')[2] for i in sellerid_full]
  for i in range(0, len(rating)): 
    try:
      float(rating[i])
    except ValueError:
      rating[i] = 0
  if rating != None:
    for i in range(0, len(rating)): 
      rating[i] = float(rating[i])/20
  reviews_full = raw_html.find_all('div', {'class' : 'rating_count'}) # The number of customer reviews
  reviews = [i.text for i in reviews_full]
  for i in range(0, len(reviews)): 
    reviews[i] = reviews[i].replace('(','').replace(')','')
    reviews[i] = int(reviews[i])
  priceUSD = [] # Price in American Dollars
  category_raw = os.path.abspath(os.path.join(filename, os.pardir))
  category_raw =  re.sub(r'^.*\\', '',category_raw).replace('drugs-','').capitalize() + '-None'
  category = category_raw.split('-')[0]
  category = [category] * len(title) # Fill column with single category value
  subcat = category_raw.split('-')[1].capitalize()
  subcat = [subcat] * len(title)
  market = ['SilkRoad2'] * len(title) # Fill column with single market name value
  date = [str(Path(filename).parents[2])[-10:]] * len(title)  # # Fill column with single date value
  df = pd.DataFrame([title,sellerid,priceUSD,priceBTC,rating,reviews,origin,destination,category,subcat,market,date]).transpose()  # Create df and transpose
  df.columns = ['Title','Sellerid','PriceUSD','PriceBTC','Rating','Reviews','Origin','Destination','Category','Subcategory','Market','Date']  # Assign column names to df
  return df

cleanhtmlsr2part3('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-08-30/categories/forgeries/items_page=2.html')
#cleanhtmlsr2part3('D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2/2014-08-23/categories/drugs-dissociatives-dioscorea/items.html')

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,[Fake Lithuanian Drivers License (new version),gorkdocs,None,1.030664,4.5,1,Undeclared,Worldwide,Forgeries,None,SilkRoad2,2014-08-30
1,REAL Genuine USA Identity Set Guide,starlight,None,0.161068,4.4,1,Undeclared,Worldwide,Forgeries,None,SilkRoad2,2014-08-30
2,Custom made stamp,gorkdocs,None,0.321026,4.5,1,Undeclared,Worldwide,Forgeries,None,SilkRoad2,2014-08-30
3,Custom HQ EU Util Bill,starlight,None,0.092700,4.4,2,Undeclared,Worldwide,Forgeries,None,SilkRoad2,2014-08-30
4,WISCONSIN Fake ID (Scans,Joseph Cosey,None,0.091748,4.4,1,United States,United States,Forgeries,None,SilkRoad2,2014-08-30
5,Holograms),ShopWithUs,None,0.214078,4.65,1,Undeclared,Worldwide,Forgeries,None,SilkRoad2,2014-08-30
6,HQ Rhode Island Fake ID. All Security Features...,iOracle,None,0.142718,2.8,0,United States,United States & Canada,Forgeries,None,SilkRoad2,2014-08-30
7,Fake ID - Illinois State - Passes: (UV/BEND/SC...,PurpleLotus,None,0.061165,4.75,0,United States,United States,Forgeries,None,SilkRoad2,2014-08-30
8,USA COUPONS - Condiments,Elchapo101,None,0.611650,3.85,1,United States,United States,Forgeries,None,SilkRoad2,2014-08-30
9,Sauces &amp; Spices Coupon Pack.,Joseph Cosey,None,20.388350,4.4,1,United States,United States,Forgeries,None,SilkRoad2,2014-08-30


In [81]:
# Create empty list to populate
data = []

# Specify file directory with folders for different scraping sessions
datedir = "D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2"

# Adds html ext to all files. Applies file cleaning function to all files from each scraping session and appends to one of two dataframes.
for scrapedate in os.listdir(datedir):
    directory = datedir +'/' + scrapedate + '/categories/'
    for foldername in os.listdir(directory):
        folder = directory + foldername # Concatenate directory path and filename for reading into cleanhtml function
#       converthtmlsr2(folder)
        for filename in os.listdir(folder):
            file = folder + '/' + filename
#            if scrapedate <= '2014-08-17':
#                filedf = cleanhtmlsr2(file)  # Extract dataframe of required variables from each file
#                data.append(filedf)  # Append empty list with each dataframe generated. Results in list of dataframes.
#                df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 
            if scrapedate >= '2014-08-30':
                filedf2 = cleanhtmlsr2part2(file)  # Extract dataframe of required variables from each file
                data.append(filedf2)  # Append empty list with each dataframe generated. Results in list of dataframes.
df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 

In [82]:
df

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,[Ray Ban Tech RB3460 001 Aviator/Flip Out/ Sun...,FoxyGirl,None,0.117934,4.55,23,China,Worldwide,Apparel,None,SilkRoad2,2014-08-30
1,Ray Ban RB3025 Aviator Classic Sunglasses Replica,FoxyGirl,None,0.073200,4.55,13,China,Worldwide,Apparel,None,SilkRoad2,2014-08-30
2,Ray Ban RB2140 Wayfarer Sunglasses Replica,FoxyGirl,None,0.071167,4.55,15,China,Worldwide,Apparel,None,SilkRoad2,2014-08-30
3,Ray Ban RB3025 112 Aviator Flash Lenses Sungla...,FoxyGirl,None,0.073200,4.55,14,China,Worldwide,Apparel,None,SilkRoad2,2014-08-30
4,Chanel Ultra Ceramic Bracelet Replica,FoxyGirl,None,0.085401,4.55,14,China,Worldwide,Apparel,None,SilkRoad2,2014-08-30
...,...,...,...,...,...,...,...,...,...,...,...,...
24,instagram services,profesorhouse,None,0.290721,4.8,0,Belgium,Undeclared,Services,None,SilkRoad2,2014-09-30
25,"500,000 High Retention Views | YouTube |",None,None,None,None,None,None,None,Services,None,SilkRoad2,2014-09-30
26,25 000 Real Twitter Followers!,None,None,None,None,None,None,None,Services,None,SilkRoad2,2014-09-30
27,"40,000 Facebook Followers Subscribers. &gt;Ch...",None,None,None,None,None,None,None,Services,None,SilkRoad2,2014-09-30


In [74]:
# Create empty list to populate
data = []

# Specify file directory with folders for different scraping sessions
datedir = "D:/Darkweb Data/dnmarchives/silkroad2.tar/silkroad2"

# Adds html ext to all files. Applies file cleaning function to all files from each scraping session and appends to one of two dataframes.
for scrapedate in os.listdir(datedir):
    directory = datedir +'/' + scrapedate + '/categories/'
    for foldername in os.listdir(directory):
        folder = directory + foldername # Concatenate directory path and filename for reading into cleanhtml function
#       converthtmlsr2(folder)
        for filename in os.listdir(folder):
            file = folder + '/' + filename
            if '2014-08-29' < scrapedate < '2014-09-01':
                filedf = cleanhtmlsr2part2(file)  # Extract dataframe of required variables from each file
#                data.append(filedf)  # Append empty list with each dataframe generated. Results in list of dataframes.
#                df = pd.concat(data)  # Concatenate list of dataframes into single dataframe 
                

##Works from 08-30

In [75]:
filedf

,Title,Sellerid,PriceUSD,PriceBTC,Rating,Reviews,Origin,Destination,Category,Subcategory,Market,Date
0,[Brazzers [LIFETIME] Vouched and Top Rated - G...,profesorhouse,None,0.020313,4.75,5,Belgium,Undeclared,Writing,None,SilkRoad2,2014-08-30
1,Professional Profiles,WickedWords,None,0.701505,4.75,7,Undeclared,Worldwide,Writing,None,SilkRoad2,2014-08-30
2,Urgent Message!,inquisition,None,0.006100,4.15,4,United States,Worldwide,Writing,None,SilkRoad2,2014-08-30
3,BangBros [LIFETIME] Vouched and Top Rated Get ...,profesorhouse,None,0.020313,4.75,1,Belgium,Undeclared,Writing,None,SilkRoad2,2014-08-30
4,Custom Copywriting,WickedWords,None,0.101667,4.75,2,Undeclared,Worldwide,Writing,None,SilkRoad2,2014-08-30
5,Improve your SEX guide pack FUCK like a pornstar,starlight,None,0.011053,4.4,1,Undeclared,Worldwide,Writing,None,SilkRoad2,2014-08-30
6,Product Listings,WickedWords,None,0.203335,4.75,1,Undeclared,Worldwide,Writing,None,SilkRoad2,2014-08-30
7,5 Unique Methods $5k+day BlackhatWhitehat,profesorhouse,None,1.626678,4.75,0,Belgium,Undeclared,Writing,None,SilkRoad2,2014-08-30
8,Academic Ghostwriting,ShroudWriter,None,0.101667,4.75,0,United States,Worldwide,Writing,None,SilkRoad2,2014-08-30
9,Setting-Up-Shop Writing Package,WickedWords,None,1.220008,4.75,0,Undeclared,Worldwide,Writing,None,SilkRoad2,2014-08-30
